In [11]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import Geohash
from build import TreeBuilding
from decision import haversine
import pickle
import gmplot
import json
from sklearn.cross_validation import train_test_split

%matplotlib inline

In [6]:
tree = pickle.load( open( "tree_250_data.p", "rb" ) )

In [3]:
def preorder(tree, points):
    if tree is not None:
        if tree.decision_point is not None:
            preorder(tree.left, points)
            preorder(tree.right, points)
            points.append(list(tree.decision_point))
            return points
        
def get_lats_lons(points):
    lats = [point[1] for point in points]
    lons = [point[0] for point in points]
    return lats, lons

In [41]:
decision_points = []
decision_points = preorder(tree.root, decision_points)

In [42]:
lats, lons = get_lats_lons(decision_points)

In [4]:
def order_left(tree, points):
    if tree is not None:
        if tree.decision_point is not None:
            #points.append(list(tree.decision_point))
            preorder(tree.left, points)
            #preorder(tree.right, points)
            points.append(list(tree.decision_point))
            return points

In [36]:
decision_points = []
decision_points = order_left(tb.root, decision_points)
decision_points = list(reversed(decision_points))

In [37]:
len(decision_points)

174

In [38]:
lats, lons = get_lats_lons(decision_points)

In [47]:
gmap = gmplot.GoogleMapPlotter(41.1496100, -8.6109900 , 12)

gmap.plot(lats[:20], lons[:20], 'cornflowerblue', edge_width=5)
#gmap.scatter(heat_lats, heat_lons, '#3B0B39', size=10, marker=False)
gmap.scatter([lats[0]], [lons[0]], '#8B0000', size=15, marker=False)
gmap.scatter(lats[1:20], lons[1:20], '#000000', size=15, marker=False)
#gmap.scatter(end_lats, end_lons, '#8B0000', size=10, marker=False)

#gmap.heatmap(heat_lats, heat_lons)

gmap.draw("plot10_new_left.html")

It seems I need two different loops:
1. To print out all nodes
2. To print out all edges

How to specify the size of the each node/edge??

It goes without saying, I need to know the number of trajectories in each node.

So, the idea: 
- the size of the node correspond to the number of trajectories in this node
- the size of the edge edge(p1,p2) correspond to the number of trajectories in p2

Осталось только понять величину максимальную

Some sort of order

In [5]:
def dec_points(traj_tree, trajectory):
    current_node = traj_tree.root
    dec_points = []  
    
    for idx, point in enumerate(trajectory):
        if len(trajectory) > (idx + 1):
            if (current_node.left == None) and (current_node.right == None):
                dec_points.append(current_node.prediction)
                break
                    
            elif (current_node.left == None) and (current_node.right != None):
                print idx, "right"
                current_node = current_node.right
                dec_points.append(current_node.decision_point)
                    
            elif (current_node.left != None) and (current_node.right == None):
                print idx, 'left'
                current_node = current_node.left
                dec_points.append(current_node.decision_point)
                    
            else:
                if haversine(point, current_node.decision_point) < current_node.radius:
                    print idx, 'left'
                    current_node = current_node.left
                    dec_points.append(current_node.decision_point)
                else:
                    current_node = current_node.right
                    print idx, 'right'
                    dec_points.append(current_node.decision_point)
                        
        else:
            dec_points.append(current_node.prediction)
            break
                
        #return [float(prediction[0]), float(prediction[1])]
    return dec_points
    

In [15]:
air_trajs = pd.read_csv('air_trajs.csv',
                        #index_col=,
                        usecols = ['POLYLINE', 'Destination'],
                        converters={'POLYLINE': lambda x: json.loads(x), 'Destination': lambda x: json.loads(x)})

trajectories = list(air_trajs['POLYLINE'])
targets = list(air_trajs['Destination'])

X_train, X_test, y_train, y_test = train_test_split(trajectories, targets, train_size=0.50, random_state = 0)

In [16]:
tb = TreeBuilding()
tb = tb.fit(X_train, y_train, 1, 10)

2164
left
699
left
416
left
60
left
3
right
57
left
4
right
53
left
3
right
50
left
1
right
49
left
15
left
1
right
14
left
1
right
13
left
1
right
12
left
1
right
11
left
1
right
10
right
34
left
3
right
31
left
5
right
26
left
1
right
25
left
2
right
23
left
1
right
22
left
1
right
21
left
1
right
20
left
18
left
1
right
17
left
1
right
16
left
2
right
14
left
13
left
1
right
12
left
3
right
9
right
1
right
2
right
356
left
14
left
13
left
2
right
11
left
10
right
1
right
1
right
342
left
2
right
340
left
76
left
3
right
73
left
3
right
70
left
2
right
68
left
2
right
66
left
2
right
64
left
2
right
62
left
11
left
6
right
5
right
51
left
3
right
48
left
35
left
24
left
1
right
23
left
1
right
22
left
2
right
20
left
1
right
19
left
1
right
18
left
1
right
17
left
16
left
2
right
14
left
1
right
13
left
1
right
12
left
11
left
2
right
9
right
1
right
1
right
11
left
9
right
2
right
13
left
2
right
11
left
2
right
9
right
264
left
3
right
261
left
10
right
251
left
4
right
247
left
4


In [17]:
def preorder_size(tree, points, p_size):
    if tree is not None:
        if tree.decision_point is not None:
            preorder_size(tree.left, points, p_size)
            preorder_size(tree.right, points, p_size)
            points.append(list(tree.decision_point))
            p_size.append(tree.num_trajs)
            return points, p_size

In [18]:
p_coord = []
p_sizes = []

p_coord, p_sizes = preorder_size(tb.root, p_coord, p_sizes)
p_coord = list(reversed(p_coord))
p_sizes = list(reversed(p_sizes))



In [19]:
gmap = gmplot.GoogleMapPlotter(41.1496100, -8.6109900 , 12)
num_points = p_sizes[0]

gmap.scatter([p_coord[0][1]], [p_coord[0][0]], '#000000', size=40, marker=False)

for i in range(1,len(p_sizes)):
    p_size = 39. * p_sizes[i] / num_points + 1
    #print p_size
    gmap.scatter([p_coord[i][1]], [p_coord[i][0]], '#3B0B39', size = p_size, marker=False)
    
gmap.draw("plot_point_airport.html")

In [14]:
#visualization of trajectories

colors = [name.encode() for name, hex in mpl.colors.cnames.iteritems()]

test_traj = X_test[400]
predicted_traj = [list(p) for p in dec_points(tree, test_traj) if p is not None]

gmap = gmplot.GoogleMapPlotter(41.1496100, -8.6109900 , 12)

lats, lons = get_lats_lons(test_traj)
gmap.plot(lats, lons, colors[0], edge_width=5)

lats, lons = get_lats_lons(predicted_traj)
gmap.plot(lats, lons, colors[2], edge_width=5)

gmap.scatter([predicted_traj[-1][1]], [predicted_traj[-1][0]], 'black', size = 20, marker=False)
gmap.scatter([test_traj[-1][1]], [test_traj[-1][0]], 'red', size = 20, marker=False)

gmap.draw("plot10_new_left.html")

0 right
1 right
2 right
3 right
4 right
5 right
6 right
7 right
8 right
9 right
10 right
11 right
12 right
13 right
14 right
15 right
16 right
17 right
18 right
19 right
20 right
21 right
22 right
23 right
24 left
25 right
26 right
27 right
28 right
29 right
30 right
31 right
